# IMDB Production Success Analysis- Part 1 

- Create a project repository, loading the official IMDB data for the requested tables, filtering out unnecessary data, and saving the filtered tables as gzip-compressed csv files (".csv.gz") in repository.

# Business Obejctive

- For this project, produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately,  this database to analyze what makes a movie successful and will provide recommendations to the stakeholder on how to make a successful movie.

# Data Source

IMDB Provides Several Files with varied information for Movies, TV Shows, Made for TV Movies, etc.
- Overview/Data Dictionary: https://www.imdb.com/interfaces/
- Downloads page: https://datasets.imdbws.com/

Focus on these files:
- title.basics.tsv.gz
- title.ratings.tsv.gz
- title.akas.tsv.gz

# Specifications

Stakeholder only wants to include information for movies based on the following specifications for Data:

- Exclude any movie with missing values for genre or runtime
- Include only full-length movies (titleType = "movie").
- Include only fictional movies (not from documentary genre)
- Include only movies that were released 2000 - 2021 (include 2000 and 2021)
- Include only movies that were released in the United States



# Import/Load libraries & Data

In [40]:
#Imported libraries

import pandas as pd 
import numpy as np
import os

In [41]:
#Loaded Data as url

basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

# Preprocessing Data

## Filtering/Cleaning- Basics

In [42]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [43]:
#Replace "\N" with np.nan

basics.replace({'\\N':np.nan}, inplace = True)

In [44]:
#Eliminate movies that are null for runtimeMinutes, genres, startYear (since can't convert nan to int)

basics.dropna(subset = ['runtimeMinutes', 'genres', 'startYear'], inplace = True)

In [47]:
#keep only titleType==movie

basics = basics.loc[basics['titleType'] == 'movie']

In [48]:
#keep startYear 2000-2022

basics['startYear'] = basics['startYear'].astype(int)
basics = basics.loc[(basics['startYear'] >= 2000) & (basics['startYear'] <=2021)]

In [49]:
#Eliminate movies that include  "Documentary" in genre (see tip below)

documentary_filter = basics['genres'].str.contains('documentary', case=False)
basics = basics[~documentary_filter]

### Making Directory 

In [50]:
#Making directory

os.makedirs('Data/',exist_ok=True) 

In [69]:
#Check if directory was made

os.listdir('Data/')

['title_basics.csv.gz', 'title_akas.csv.gz', 'title_ratings.csv.gz']

## Filtering/Cleaning- Akas

In [52]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [53]:
#Replace "\N" with np.nan

akas.replace({'\\N':np.nan}, inplace = True)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [54]:
#keep only US entries.

akas = akas.loc[akas['region'] =='US']

In [55]:
#Filter the basics table down to only include the US by using the filter akas dataframe

us_filter = basics['tconst'].isin(akas['titleId'])
us_filter

34805       True
61119       True
67672       True
77968      False
86806       True
           ...  
8971390     True
8971399     True
8971438    False
8971483     True
8971567    False
Name: tconst, Length: 135950, dtype: bool

In [56]:
#Filtering basics dataframe 

basics = basics[us_filter]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34805,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61119,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67672,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86806,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
91077,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0,2020,NaN,74,"Horror,Music,Thriller"
...,...,...,...,...,...,...,...,...,...
8970854,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
8971250,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
8971390,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
8971399,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,90,"Action,Adventure,Thriller"


### Saving File- Basics & Akas

In [57]:
#Save dataframe to file

basics.to_csv("Data/title_basics.csv.gz", compression='gzip', index=False)

In [58]:
#Checked saved file 

basics = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0,2020,NaN,74,"Horror,Music,Thriller"


In [59]:
#Checked info of basics file

basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79325 entries, 0 to 79324
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          79325 non-null  object 
 1   titleType       79325 non-null  object 
 2   primaryTitle    79325 non-null  object 
 3   originalTitle   79325 non-null  object 
 4   isAdult         79325 non-null  int64  
 5   startYear       79325 non-null  int64  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  79325 non-null  int64  
 8   genres          79325 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 5.4+ MB


In [60]:
#Save dataframe to file

akas.to_csv("Data/title_akas.csv.gz", compression='gzip', index=False)

In [61]:
#Checked saved file 

akas = pd.read_csv("Data/title_akas.csv.gz", low_memory=False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [62]:
#Checked info of basics file

akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326061 entries, 0 to 1326060
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1326061 non-null  object 
 1   ordering         1326061 non-null  int64  
 2   title            1326061 non-null  object 
 3   region           1326061 non-null  object 
 4   language         3564 non-null     object 
 5   types            1025775 non-null  object 
 6   attributes       44112 non-null    object 
 7   isOriginalTitle  1324686 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 80.9+ MB


## Filtering/Cleaning- Ratings

In [63]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1884
1,tt0000002,5.9,250
2,tt0000003,6.5,1669
3,tt0000004,5.8,163
4,tt0000005,6.2,2491


In [64]:
#Replace "\N" with np.nan

ratings.replace({'\\N':np.nan}, inplace = True)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1884
1,tt0000002,5.9,250
2,tt0000003,6.5,1669
3,tt0000004,5.8,163
4,tt0000005,6.2,2491


### Saving File

In [65]:
#Save dataframe to file

ratings.to_csv("Data/title_ratings.csv.gz", compression='gzip', index=False)

In [66]:
#Checked saved file 

ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1884
1,tt0000002,5.9,250
2,tt0000003,6.5,1669
3,tt0000004,5.8,163
4,tt0000005,6.2,2491


In [67]:
#Checked info of basics file

ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1251067 entries, 0 to 1251066
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1251067 non-null  object 
 1   averageRating  1251067 non-null  float64
 2   numVotes       1251067 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.6+ MB
